In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import os

In [ ]:
os.chdir(r'D:\Corona\Corona_Analysis')
os.getcwd()

In [ ]:
url="https://www.thehindu.com/sport/cricket/coronavirus-former-cricketer-sukhvinder-singh-recounts-his-rescue-mission-to-italy/article31167345.ece"

In [ ]:
url = "https://www.thehindu.com/topic/coronavirus/"

In [ ]:
html = requests.get(url)
soup=bs(html.content,"html.parser")

In [ ]:
# get page link
page_link=soup.select("[class='pagination']")
page_link[0].find_all('a')[0]['href']

# last page number on this topic
last_link=page_link[0].find_all('a')[-1]['href']
last_page=re.search("page=\d*",last_link).group(0)
last_page_number=int(last_page.replace("page=",""))
last_page_number

In [ ]:
# iterarte over all pages in the range to create and get the link
upd_url=[]
title=[]
link=[]
page_number=[]
for i in range(1,last_page_number+1):
    add_page_number=url+"?page="+str(i)
    #upd_url.append(add_page_number)
    page_url=add_page_number
    print(page_url)
    html_page = requests.get(page_url)
    soup_page=bs(html_page.content,"html.parser")
    all_h3=soup_page.find_all("h3")

    for j in all_h3:
        try:
            title.append(j.find('a').get_text().replace("\n",""))
            link.append(j.find('a')['href'])
            page_number.append(i)
        except AttributeError:
            print("skip")
    

In [ ]:
df=pd.DataFrame({'page_number':page_number,'title':title,"link":link})

In [ ]:
df.to_excel('link.xlsx')

In [ ]:
# html_value = requests.get(link_value)
# link_soup=bs(html_value.content,"html.parser")
# article = link_soup.select("[class~=article]")
# at_c=article[0].select("[class*='author-container hidden-xs']")[0]
# at_c.find_all('span')[0].get_text().replace("\n","")
# print(at_c)

In [ ]:
section_list = []
title_list = []
content_list = []
author_list = []
creation_date_list = [] 
counter = 1140
link_pinged=[]
for link_value in link[1140:]:
    try:
        
        print(counter)
        try:
            html_value = requests.get(link_value)
        except ValueError:
            html_value = None
        link_soup=bs(html_value.content,"html.parser")
        article = link_soup.select("[class~=article]")
        sec=article[0].select("[class~=section-name]")
        try:
            section=sec[0].get_text().replace("\n","")
        except IndexError:
            section = None
        title = article[0].select("[class~=title]")
        try:
            title = title[0].get_text().replace("\n","")
        except IndexError:
            title = None
        body = article[0].select("[id*='content-body-']")
        try:
            content=body[0].get_text().replace("\n","")
        except IndexError:
            content = None

        ut_c = article[0].select("[class*='ut-container']")[0]
        #author_c=article[0].select("[class*='author-container*']")[0]
        at_c=article[0].select("[class*='author-container hidden-xs']")[0]
        try:
            author_name = at_c.find_all('span')[0].get_text().replace("\n","")
        except IndexError:
            author_name = None
        #print(author_name)
        #author_name = ut_c.find_all('span')[0].get_text().replace("\n","")
        #print(author_name)
        try:
            creation_date = ut_c.find_all('span')[1].get_text().replace("\n","")
        except IndexError:
            creation_date = None
        #print(creation_date)
        # update the list 
        section_list.append(section)
        title_list.append(title)
        content_list.append(content)
        author_list.append(author_name)
        creation_date_list.append(creation_date)
        link_pinged.append(link_value)
        counter = counter + 1
    except ChunkedEncodingError:
        print(link_value)
        counter = counter + 1

In [ ]:
data_collected = {'Author':author_list,'Date':creation_date_list,'Title':title_list,'Section':section_list, 'content': content_list, 'url':link_pinged}

In [ ]:
data_collected

In [ ]:
final_df=pd.DataFrame(data_collected)

final_df.to_excel("The_hindu_extract_corona_remaining.xlsx",index=False)

In [ ]:
final_df

In [ ]:
print(len(author_list))
print(len(creation_date_list))
print(len(title_list))
print(len(section_list))
print(len(content_list))
print(len(link[:660]))

title_list


In [ ]:
{'Author':author_list,'Date':creation_date_list,'Title':title_list,'Section':section_list, 'content': content_list, 'url':link}